# CORDIS Data EDA

Reference data from [here](https://data.europa.eu/euodp/data/dataset/cordisref-data).

## Preamble

In [ ]:
%run notebook_preamble.ipy

pd.set_option('max_columns', 99)

In [ ]:
import seaborn as sns
import string

from eu_funding.visualization.visualize import pdf_cdf
# from src.visualization.visualize import pdf_cdf

In [ ]:
sns.set_context??

In [ ]:
sns.set_context('notebook')

## Data

### Projects

Each funding programme has a projects dataset, which contain research projects funded by EC programmes.
- `rcn`
- `id`
- `acronym`
- `status`
- `programme`
- `topics`
- `frameworkProgramme`
- `title`
- `startDate`
- `endDate`
- `projectUrl`
- `objective`
- `totalCost`
- `ecMaxContribution`
- `call`
- `fundingScheme`
- `coordinator`
- `coordinatorCountry`
- `participants`
- `participantCountries`
- `subjects`

### H2020

In [ ]:
h2020_projects_df = pd.read_csv(
    os.path.join(cordis_h2020_path, 'cordis-h2020projects.csv'),
    sep=';',
    encoding='iso-8859-1',
    parse_dates=['startDate', 'endDate'],
    infer_datetime_format=True,
    decimal=','
)
h2020_orgs_df = pd.read_csv(
    os.path.join(cordis_h2020_path, 'cordis-h2020organizations.csv'),
    sep=';',
    encoding='iso-8859-1',
    decimal=',',
)
h2020_reports_df = pd.read_csv(
    os.path.join(cordis_h2020_path, 'cordis-h2020reports.csv'),
)
h2020_pi_df = pd.read_excel(
    os.path.join(cordis_h2020_path, 'cordis-h2020-erc-pi.xlsx'),
    sheet_name='PI'
)
h2020_fellows_r1_df = pd.read_excel(
    os.path.join(cordis_h2020_path, 'cordis-h2020-msca-fellows.xls'),
    sheet_name='Report 1',
    header=3
).set_index('Project Number')
h2020_fellows_r2_df = pd.read_excel(
    os.path.join(cordis_h2020_path, 'cordis-h2020-msca-fellows.xls'),
    sheet_name='Report 2'
).set_index('projectId')
h2020_fellows_df = h2020_fellows_r1_df.join(h2020_fellows_r2_df, how='outer').reset_index().rename(columns={'index': 'projectId'})

In [ ]:
h2020_projects_df.head(1)

In [ ]:
h2020_projects_df.shape

In [ ]:
h2020_reports_df.head(1)

In [ ]:
h2020_reports_df.shape

In [ ]:
h2020_orgs_df.head(1)

In [ ]:
h2020_orgs_df.shape

In [ ]:
h2020_pi_df.head(1)

In [ ]:
h2020_fellows_df.head(1)

### FP7

In [ ]:
fp7_projects_df = pd.read_csv(
    os.path.join(cordis_fp7_path, 'cordis-fp7projects.csv'),
    sep=';',
    encoding='iso-8859-1',
    parse_dates=['startDate', 'endDate'],
    infer_datetime_format=True,
    decimal=','
)
fp7_orgs_df = pd.read_csv(
    os.path.join(cordis_fp7_path, 'cordis-fp7organizations.csv'),
    sep=';',
    encoding='iso-8859-1',
    decimal=',',
)
fp7_reports_df = pd.read_csv(
    os.path.join(cordis_fp7_path, 'cordis-fp7reports.csv'),
).drop('Unnamed: 16', axis=1)

In [ ]:
fp7_projects_df.head(1)

In [ ]:
fp7_orgs_df.head(1)

In [ ]:
fp7_projects_df[fp7_projects_df['id'] == 240271]

### Combining Programme Datasets

In [ ]:
cordis_projects_df = pd.concat([h2020_projects_df, fp7_projects_df])
cordis_reports_df = pd.concat([h2020_reports_df, fp7_reports_df])
cordis_orgs_df = pd.concat([h2020_orgs_df, fp7_orgs_df])

### Reference Data

#### Activity Types

In [ ]:
cordis_activity_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-organizationActivityType.csv'),
    sep=';'
)

In [ ]:
cordis_activity_ref_df

#### Countries

In [ ]:
cordis_countries_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-countries.csv'),
    sep=';'
)

In [ ]:
cordis_countries_ref_df.head(10)

#### Topics

In [ ]:
cordis_topics_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-H2020topics.csv'),
    sep=';'
)

In [ ]:
cordis_topics_ref_df.head()

#### H2020 Programmes

In [ ]:
cordis_h2020_programmes_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-H2020programmes.csv'),
#     sep=';'
)

In [ ]:
cordis_h2020_programmes_ref_df.head()

#### Funding Schemes

In [ ]:
cordis_funding_schemes_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-projectFundingSchemeCategory.csv'),
    sep=';'
)

In [ ]:
cordis_funding_schemes_ref_df.head()

#### SIC Codes

In [ ]:
cordis_sic_codes_ref_df = pd.read_csv(
    os.path.join(cordis_ref_path, 'cordisref-sicCode.csv'),
    sep=';'
)

In [ ]:
cordis_sic_codes_ref_df[cordis_sic_codes_ref_df['language'] == 'en'].head()

## Exploration

### Basic

### Project Funding, Participants and Timing

#### Overall Funding Distribution

In [ ]:
fig, ax = plt.subplots(
    ncols=2,
    figsize=(11, 4),
)

pdf_cdf(
    ax[0], np.log10(cordis_projects_df['ecMaxContribution'][~pd.isnull(cordis_projects_df['ecMaxContribution'])]),
    bins=100
)
# pdf_cdf(
#     ax[1],
#     np.log10(cordis_projects_df['totalCost'][~pd.isnull(cordis_projects_df['totalCost'])]),
#     bins=100
# )
x = cordis_projects_df['ecMaxContribution'] / cordis_projects_df['totalCost']
x = x[~np.isnan(x)]
ax[1].hist(
    x,
    bins=25
)
ax[0].set_xlabel('EU Funding (log$_{10}$ EUR)')
# ax[1].set_xlabel('Toal Cost (log$_{10}$ EUR)')
ax[1].set_xlabel('EU Funding / Total Cost')
ax[1].set_ylabel('Frequency')

# for n, a in enumerate(ax):
#     a.text(-.2, 1.1, string.ascii_lowercase[n], transform=a.transAxes, 
#             size=14)

plt.tight_layout()
plt.show()

The distribution of EC funding across all projects shows a spiky profile with an underlying bimodal characteristic. This highlights that there are two broad funding categories; those in the order of €100k and those in the order of €1m. Within this distribution there are particularly high frequencies of individual grants centred on €50k, €100k, €250k, €2m, €2.5m, €3m and €5m amounts.

We can also see that the overall distribution of the total cost of EC funded projects mirrors the EC contribution.

Looking at the ratio between EC project contributions and total project costs, we can see that in the vast majority of cases, 100% of funds come from the EC. Other projects recieve between 0 and 85% of their funds from other sources, with the 20 to 25% being the most common fraction contributed from third parties.

#### Funding and Number of Projects By Institution

In [ ]:
orgs_name_group = cordis_orgs_df.groupby('name')

org_ec_contributions = np.log10(orgs_name_group.sum()['ecContribution'])
org_ec_contributions[np.isinf(org_ec_contributions)] = 0

org_n_projects = np.log10(orgs_name_group.count()['projectID'].values)

ec_orgs_projects_df = cordis_orgs_df.merge(
    cordis_projects_df, 
    left_on='projectID',
    right_on='id'
)[['ecContribution', 'ecMaxContribution']]
fraction_funding_org = ec_orgs_projects_df['ecContribution'] / ec_orgs_projects_df['ecMaxContribution']
fraction_funding_org = fraction_funding_org[~np.isnan(fraction_funding_org)].values

In [ ]:
fig, ax = plt.subplots(ncols=3, figsize=(16, 4))
pdf_cdf(
    ax[0],
    org_ec_contributions.values,
    bins=100
)
pdf_cdf(
    ax[1],
    org_n_projects,
    bins=50
)
pdf_cdf(
    ax[2],
    fraction_funding_org,
    bins=50
)
ax[0].set_xlabel('EU Funding (log$_{10}$ EUR)')
ax[1].set_xlabel('Number of Projects per Organisation (log$_{10}$)')
ax[2].set_xlabel('Fraction of EC Project Recieved by Institutions')
ax[2].set_xlim((0, 1.05))
plt.tight_layout()
plt.show()

If we look at the distribution of amounts awarded in projects by the EC grouped by institution, we can see several interesting properties. First that around 15% of institutions that are named on projects recieve no EC funding. Second, that we see a sharp normal-like distribution centred around the order of €100k and tails extending between the orders of €1k and €100m. Finally we can see within this distribution a sharp peak of institutions recieving around €50k. As we saw a spike for this amount in the overall distribution of project funding, we could infer that this is from an EC programme to award grants of €50k to individual researchers or institutions.

Looking at the number of projects each organisation participates in, we are presented with a distribution that stretches from insitutions involved in only 1 project to those with their fingers in thousands of pies. In fact, around 60% of institutions in the dataset are involved in only one project. It's important to bear in mind that these may be solo projects or collaborations.

Finally, we can dissect the fraction of funding recieved by institutions on each project. As we would expect from the previous plots, 15% of projects see a single institution getting 100% of the project funding. However, more generally we can identify that in the majority of cases, each institution involved in a project gets less than 20% of the funds. This leads to two questions: 

1. Is funding generally spread equally between institutions within a project?
2. Does the spike of institutions recieving 100% of project funding point to a significant number of collaborative projects where one or more institutions recieves nothing?

#### Distribution of Funding Between Collaborators

In [ ]:
def all_particpants(coordinator, participants):
    if pd.isnull(participants):
        return coordinator
    else:
        if coordinator in participants:
            return participants
        else:
            return participants + ';' + coordinator

In [ ]:
cordis_projects_df['allParticipants'] = cordis_projects_df.apply(
    lambda x: all_particpants(x['coordinator'], x['participants']),
    axis=1
)
cordis_projects_df['participant_count'] = [len(s.split(';')) for s in cordis_projects_df['allParticipants']]

In [ ]:
ec_orgs_projects_df = cordis_orgs_df.merge(
    cordis_projects_df, 
    left_on='projectID',
    right_on='id'
)[['projectID', 'ecContribution', 'ecMaxContribution', 'participant_count']]
ec_orgs_projects_df = ec_orgs_projects_df[ec_orgs_projects_df['participant_count'] > 1]
fraction_funding_org = ec_orgs_projects_df['ecContribution'] / ec_orgs_projects_df['ecMaxContribution']
fraction_funding_org = fraction_funding_org[~np.isnan(fraction_funding_org)].values

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))
pdf_cdf(
    ax[0][0],
    cordis_projects_df['participant_count'],
    bins=160
)
pdf_cdf(
    ax[0][1],
    ec_orgs_projects_df.groupby('projectID')['participant_count'].max(),
    bins=160
)
pdf_cdf(
    ax[1][0],
    fraction_funding_org,
    bins=100
)
pdf_cdf(
    ax[1][1],
    1 / ec_orgs_projects_df.groupby('projectID')['participant_count'].max(),
    bins=50
)
ax[1][1].set_xlim((0, 1))
ax[0][0].set_xlabel('Number of Participants')
ax[0][1].set_xlabel('Number of Participants\n(projects with more than one particpant)')
ax[1][0].set_xlabel('Institutional EC Contribution / Max EC Contribution \n (projects with more than one participant)')
ax[1][1].set_xlabel('Expected Funding Fractions for Even Funding Distributions')
plt.tight_layout()
plt.show()

At least 60% of projects have only one participant, the coordinator, while 95% of projects have 20 particpants or fewer. The overall distribution stretches to include at least one project with over 170 participant institutions.

If we exclude projects with only one participant, we can see that around 20% of the remaining projects are two party collaborations, and the rest form a smooth distribution centered on the median value of 9 participants.

Finally, we can calculate the fraction of fraction of project funding each institution in a collaborative project was awarded. We still see a small number (~5%) of projects where a single party obtained 100% of the funds, however the mean value among the rest of the distribution is 0.11. It is surprising that there is not a large peak at 0.5 that corresponds to the proportion of projects with two participants.

The final figure shows the expected distribution of fractional funding if funds were split evenly between participants. 

In [ ]:
max_project_contribution = ec_orgs_projects_df.groupby('projectID')['ecContribution'].max()
min_project_contribution = ec_orgs_projects_df.groupby('projectID')['ecContribution'].min()

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

normalised_contribution_difference = ((max_project_contribution - min_project_contribution) / 
         ec_orgs_projects_df.groupby('projectID')['ecMaxContribution'].max())

pdf_cdf(ax[0][0],
        normalised_contribution_difference,
        bins=50
       )
ax[0][0].set_xlabel('Highest and Lowest Contribution Difference\n Normalised by Total Funding')
pdf_cdf(ax[0][1],
        np.log10(max_project_contribution / min_project_contribution),
        bins=50
       )
ax[0][1].set_xlabel('Funding of Highest Funded Participant / Lowest Funded Participant (log$_{10}$)')
ax[1][0].hexbin(
    ec_orgs_projects_df.groupby('projectID')['participant_count'].max(),
    normalised_contribution_difference,
    bins='log',
    mincnt=1
)
ax[1][0].set_xlabel('Number of Participants')
ax[1][0].set_ylabel('Highest and Lowest Contribution Difference\n Normalised by Total Funding')
ax[1][1].hexbin(
    ec_orgs_projects_df['ecMaxContribution'],
    ec_orgs_projects_df['ecContribution'] / ec_orgs_projects_df['ecMaxContribution'],
    bins='log',
    mincnt=1,
    xscale='log'
)
ax[1][1].set_xlabel('Max EC Funding Contribution')
ax[1][1].set_ylabel('Institutional Contribution / Max Contribution')
plt.tight_layout()
plt.show()

One way to assess distribution of funding among collaborative projects is the difference between the highest and lowest allocations of funding to institutions within a project. Normalising this by the total project funding, we can see that the difference for at least 50% of projects is around 20% of the total funding.

To get a sense of the scale of the difference within projects, we can also plot the distribution of the ratio between the highest funded and lowest participants in a project. We can see that in some cases, the highest recipient is awarded hundreds or thousands of times more than the lowest.

If we look at the impact that the number of participants has on the split of funding between them, we see a broad distribution with lower numbers of collaborators, although the modal value appears to be an even division of resources. As the number of participants increases, we can see the distribution narrow until it stabilises around the mean relative difference of 0.24.

A similar trend is seen with the relationship between normalised relative funding difference and the total EC contribution. As projects are awarded more money, they generally split the funds more equally. 

Although 60% of projects are carried out by a single institution, the distribution of funds among the other 40% is potentially a very useful metric as it might indicate the relative levels of input each particpant has on a piece of research. Here, we have shown a few potential ways of measuring the distribution and identifying interesting situations, however there are of course many other potential paths to investigate for metrics used to convey this information. 

#### Total Funding Over Time

In [ ]:
def split_date(df, col):
    '''split_date
    
    Args:
        df (pandas.DataFrame): A dataframe
        col (str): A column containing datetimes
        
    Returns
        df (pandas.DataFrame): The input dataframe with year, month and
            day values for `col` in new columns.
    '''
    df[col + '_year'] = df[col].dt.year
    df[col + '_month'] = df[col].dt.month
    df[col + '_day'] = df[col].dt.day
    return df

In [ ]:
cordis_projects_df = split_date(cordis_projects_df, 'startDate')
cordis_projects_df_start_year = cordis_projects_df.groupby('startDate_year').describe().reset_index()

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 3.7))
# ax[0].plot(
#     cordis_projects_df_start_year['startDate_year'],
#     cordis_projects_df_start_year['totalCost']['count'] * cordis_projects_df_start_year['totalCost']['mean'],
#     label='Total Cost'
# )
ax[0].plot(
    cordis_projects_df_start_year['startDate_year'],
    cordis_projects_df_start_year['ecMaxContribution']['count'] * cordis_projects_df_start_year['ecMaxContribution']['mean'],
    label='EC Funding',
    linewidth=2
)
ax[1].plot(
    cordis_projects_df_start_year['startDate_year'],
    cordis_projects_df_start_year['id']['count'],
    linewidth=2
)
ax[0].scatter(
    cordis_projects_df_start_year['startDate_year'],
    cordis_projects_df_start_year['ecMaxContribution']['count'] * cordis_projects_df_start_year['ecMaxContribution']['mean'],
    label='EC Funding'
)
ax[1].scatter(
    cordis_projects_df_start_year['startDate_year'],
    cordis_projects_df_start_year['id']['count']
)

# ax[2].scatter(
#     cordis_projects_df_start_year['totalCost']['count'],
#     cordis_projects_df_start_year['totalCost']['count'] * cordis_projects_df_start_year['totalCost']['mean'],
# )
ax[0].set_ylabel('Total Funding (EUR)')
ax[0].set_xlabel('Project Start Year')
ax[1].set_ylabel('N Projects')
ax[1].set_xlabel('Project Start Year')
# ax[2].set_xlabel('N Projects')
# ax[2].set_ylabel('Total Funding (EUR)')
# handles, labels = ax[0].get_legend_handles_labels()
# ax[0].legend(handles, labels)
plt.tight_layout()
plt.show()

The total funding awarded to projects by their starting year shows that generally between €4m and €12m have been awarded annually. The lower figure in 2007 may indicate overlap with the previous funding programme. There is also a dip in 2014, which is when the H2020 programme began, so this is perhaps indicative of the previous projects finishing and the new wave of projects not having yet started. Finally, we are presented with a decline in the amount in 2019 and 2020. This is likely due to the H2020 programme ending in 2020, and therefore fewer, shorter projects are due to start in those last two years.

The number of projects starting each year closely mirrors the trend in the amount spend.

Indeed, if we plot the number total funding awarded against the number of projects starting each year, we can see a highly linear relationship.

In [ ]:
from matplotlib.ticker import MaxNLocator

In [ ]:
sy = cordis_projects_df['startDate_year'][~pd.isnull(cordis_projects_df['startDate_year'])].astype(int)
ec_mc = cordis_projects_df['ecMaxContribution'][~pd.isnull(cordis_projects_df['startDate_year'])]

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))
ax = sns.violinplot(
    sy,
    np.log10(ec_mc),
    color='C0'
)
ax.set_xlabel('Project Start Year')
ax.set_ylabel('EC Contribution (log$_{10}$ EUR)')

ax.set_xlim((0.5, 11.5))
plt.show()

A violin plot can show us the change in funding allocations over the start year of the projects. We can see between 2008 and 2014 the more or less bimodal distribution seen earlier, with peaks centred around €250k and €2.5m. From 2015 onwards, we can observe the appearance of a third peak below €100k. This is perhaps a new H2020 intitiative to fund early career researchers.

#### Number of Countries

In [ ]:
def number_of_entities(s):
    ents =  s.str.split(';')
    counts = []
    for e in ents:
        if type(e) == list:
            counts.append(len(e))
        else:
            counts.append(np.nan)
    return counts

cordis_projects_df['allCountries'] = cordis_projects_df.apply(
    lambda x: all_particpants(x['coordinatorCountry'], x['participantCountries']),
    axis=1
).fillna('NA')
cordis_projects_df['countries_count'] = [len(s.split(';')) for s in cordis_projects_df['allCountries']]

In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))

pdf_cdf(ax[0][0],
    cordis_projects_df['countries_count'],
    bins=40
)
pdf_cdf(ax[0][1],
    cordis_projects_df['participant_count'],
    bins=100
)

pdf_cdf(ax[1][0],
    cordis_projects_df['countries_count'][cordis_projects_df['countries_count'] > 1],
    bins=45
)
pdf_cdf(ax[1][1],
    cordis_projects_df['participant_count'][cordis_projects_df['participant_count'] > 1],
    bins=165
)

ax[0][0].set_xlabel('N Participant Countries')
ax[0][1].set_xlabel('N Participant Institutions')
ax[1][0].set_xlabel('N Participant Countries (Collaborative Projects)')
ax[1][1].set_xlabel('N Participant Institutions (Collaborative Projects)')

plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 4), gridspec_kw={'width_ratios': [0.45, 0.55]})

participant_count_group_country_count = cordis_projects_df.groupby('countries_count')['participant_count'].mean()

ax[0].scatter(
    participant_count_group_country_count.index,
    participant_count_group_country_count,
    color='C6',
    label='Mean N Participants'
)
handles, labels = ax[0].get_legend_handles_labels()
ax[0].legend(handles, labels)
ax[0].hexbin(
    cordis_projects_df['countries_count'],
    cordis_projects_df['participant_count'],
    bins='log',
    mincnt=1,
    gridsize=45,
)
ax[0].scatter(
    participant_count_group_country_count.index,
    participant_count_group_country_count,
    color='C6',
    label='Mean'
)

hb = ax[1].hexbin(
    cordis_projects_df['participant_count'],
    cordis_projects_df['ecMaxContribution'],
    bins='log',
    yscale='log',
    mincnt=1,
    gridsize=45
)
cb = fig.colorbar(hb, ax=ax[1])

ax[0].set_xlabel('N Participant Countries')
ax[0].set_ylabel('N Participant Institutions')
ax[1].set_xlabel('N of Participant Institutions')
ax[1].set_ylabel('EC Funding (EUR)')

plt.tight_layout()
plt.show()

The distribution of the number of countries involved in each project resembles that of the number of participants involved. However, the scale is different, with a maximum of around 45 countries involved in a single project (compared to up to ~175 participants). As expected, around 60% of projects involve just one country.

The funding distribution across projects with small numbers of countries involved resembles that of the overall distribution, and projects ranging across the whole spectrum of funding amounts. There are a significant number of projects carried out between 5 - 10 countries that recieve multiple millions of Euros. As the number of participant countries increases, the project elligibility for smaller grants appears to disappear. For example, no project with more than 10 participants recieved a grant of less than €200k.

There is a roughly linear relationship between the number of countries participating and the number of institutions involved in a project. The modal value is of a course a single country and organisation. We can see that the maximum number of institutions involved in a project can often exceed the number of countries, meaning that there must be multiple institutions from a single nationality involved in a proportion of projects.

Plotting the mean number of organisations on a project against the number of countries involved, we can see that the number of participants is roughly equal to twice the number of countries up until 20 nations are involved. After this, we see a tapering off of this trend.

#### Project Duration

In [ ]:
duration = cordis_projects_df['endDate'] - cordis_projects_df['startDate'] 

In [ ]:
fig, ax = plt.subplots(ncols=2, figsize=(10, 4), gridspec_kw={'width_ratios': [0.45, 0.55]})

pdf_cdf(ax[0], duration.dt.days / 365.25, bins=50)
hb = ax[1].hexbin(
    duration.dt.total_seconds() / (365.25 * 24 * 3600),
    cordis_projects_df['ecMaxContribution'],
    bins='log',
    yscale='log',
    mincnt=1,
    gridsize=45
)
# ax[2].hexbin(
#     cordis_projects_df['participant_count'],
#     duration.dt.total_seconds() / (365.25 * 24 * 3600),
#     bins='log',
#     mincnt=1,
#     gridsize=50
# )
ax[0].set_xlabel('Duration (years)')
ax[1].set_ylabel('EU Funding (EUR)')
ax[1].set_xlabel('Duration (Years)')
cb = fig.colorbar(hb, ax=ax[1])
# cb.set_label('counts')
# ax[2].set_xlabel('Number of Participant Countries')
# ax[2].set_ylabel('Duration (Years)')
plt.tight_layout()
plt.show()

Project durations cover a span from a few months to several years, with major peaks at 2, 3, 4 and 5 years, and a mean length of 3.28 years.

Typically longer projects are awarded more funding. A project designed to run for 5 years might recieve up to 10 times more funding than one set to last for just 2 years.

There is no significant trend between the number of participating countries and the duration of a project.

#### Country Level Funding Distributions

In [ ]:
cordis_orgs_df_by_country = cordis_orgs_df.groupby('country')
cordis_projects_df_by_country = cordis_projects_df.groupby('coordinatorCountry')
topn = 20

fig, ax = plt.subplots(ncols=3, figsize=(16, 4))
cordis_projects_df_by_country.count()['id'].sort_values()[-topn:].plot.barh(
    color='C0',
    ax=ax[0])
cordis_orgs_df_by_country.count()['id'].sort_values()[-topn:].plot.barh(
    color='C0',
    ax=ax[1])
cordis_orgs_df_by_country.sum()['ecContribution'].sort_values()[-topn:].plot.barh(color='C0', ax=ax[2])
ax[0].set_xlabel('N Projects')
ax[0].set_ylabel('Coordinator Country')
ax[1].set_xlabel('N Project Particpants')
ax[1].set_ylabel('Country')
ax[2].set_xlabel('EC Funding (EUR)')
ax[2].set_ylabel('Country')
plt.show()

The UK is the coordinating country of the most awards, accounting for almost 9,000 projects. Only Germany, Spain, France, Italy and the Netherlands also coordinate more than 2,000 projects each.

However, when this is broken down by the number of times an institution from a country has participated in a project, we can see that Germany just overtakes the UK, with both having over 30,000 institutional involvements in a project. The top 6 countries are again the same.

The same 6 countries occupy the top spots when it comes to the total amounts of funding recieved from the EC. Again, Germany and the UK are very close, both having gained just over €14bn, and France is the only other country to have been awarded a total greater than €10bn.

#### Collaborations

In [ ]:
from itertools import combinations, repeat
from collections import defaultdict

In [ ]:
country_collaborations = defaultdict(list)

for participants, coordinator in zip(cordis_projects_df['participantCountries'][~pd.isnull(cordis_projects_df['participantCountries'])],
                    cordis_projects_df['coordinatorCountry']):
    p = participants.split(';')
    for pa, co in zip(p, repeat(coordinator, len(p))):
        if co != pa:
            country_collaborations['Coordinator Country'].append(co)
            country_collaborations['Participant Country'].append(pa)
            country_collaborations['count'].extend([1])

In [ ]:
country_counts_df = pd.DataFrame(country_collaborations)

In [ ]:
country_counts_pivot = country_counts_df.pivot_table(
    index='Coordinator Country',
    values='count',
    columns='Participant Country',
#     aggfunc=lambda x: np.log(np.sum(x)),
    aggfunc=np.sum
)

In [ ]:
sort_order = cordis_projects_df['coordinatorCountry'].value_counts().index.values
sort_index = [s for s in sort_order if s in country_counts_pivot.index]
sort_cols = [s for s in sort_order if s in country_counts_pivot.columns]

In [ ]:
country_counts_pivot = country_counts_pivot.reindex(
    sort_index
)[sort_cols]

In [ ]:
from matplotlib.colors import LogNorm

In [ ]:
fig, ax = plt.subplots(figsize=(20, 11))
ax = sns.heatmap(country_counts_pivot,
#                  norm=LogNorm(vmin=country_counts_pivot.min().min(), vmax=country_counts_pivot.max().max())
                )
plt.show()

In [ ]:
cordis_projects_df.head()

In [ ]:
from collections import Counter
from itertools import chain

In [ ]:
c_participants = Counter(chain(*cordis_projects_df['allParticipants'].str.split(';')))

In [ ]:
len(c_participants)

In [ ]:
cordis_orgs_df.head()

In [ ]:
cordis_reports_df['projectID'].value_counts()

In [ ]:
cordis_reports_df[cordis_reports_df['projectID'] == 262948]['article'].iloc[0]

In [ ]:
i = 500
pid = cordis_reports_df.iloc[i]['projectID']
print(pid)
print(cordis_reports_df.iloc[i]['article'])

In [ ]:
cordis_projects_df[cordis_projects_df['id'] == 287600].iloc[0]['objective']

## Orgs

In [ ]:
cordis_activity_ref_df

In [ ]:
activity_type_mapping = {
    'HES': 'Higher or Secondary Education Establishments',
    'REC': 'Research Organisations',
    'PRC': 'Private for-profit entities (excl. Educational)',
    'PUB': 'Public bodies (excl. Research and Educational)',
    'OTH': 'Other'
}

In [ ]:
cordis_orgs_df['activityTypeTitle'] = cordis_orgs_df['activityType'].map(activity_type_mapping)

In [ ]:
cordis_orgs_df.drop_duplicates('name')['activityTypeTitle'].value_counts()

In [ ]:
cordis_orgs_df['activityTypeTitle'].value_counts()

In [ ]:
fig, ax = plt.subplots(ncols=2, sharey=True, figsize=(12, 4))

cordis_orgs_df.drop_duplicates('name')['activityTypeTitle'].value_counts().plot(kind='barh', color='C0', ax=ax[0])
ax[0].invert_yaxis()

cordis_orgs_df['activityTypeTitle'].value_counts()[
    cordis_orgs_df.drop_duplicates('name')['activityTypeTitle'].value_counts().index
].plot(kind='barh', color='C0', ax=ax[1])
ax[1].invert_yaxis()

ax[0].set_ylabel('Institution Type')
ax[0].set_xlabel('N Unique Institutions')
ax[1].set_xlabel('N Institutions Participating in Project')

plt.tight_layout()
plt.show()

In [ ]:
x = axs[i]

In [ ]:
x.get_ylim()

In [ ]:
fig, axs = plt.subplots(nrows=5, figsize=(12, 10))

axs = axs.ravel()

for i, code in enumerate(cordis_activity_ref_df['Code']):
    x = cordis_orgs_df[cordis_orgs_df['activityType'] == code]['name']
    x.value_counts()[:10].plot(kind='barh', color='C0', ax=axs[i])
    axs[i].invert_yaxis()
    pos_x = axs[i].get_xlim()[1]
    axs[i].text(0.85 * pos_x, 8.5, code, 
#                 transform=ax.transAxes, 
#             size=20, weight='bold'
               )
axs[-1].set_xlabel('N Projects')

plt.tight_layout()
plt.show()

In [ ]:
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12,6))

axs = axs.ravel()

for i, (l, g) in enumerate(cordis_orgs_df.groupby('activityType')):
    pdf_cdf(
        axs[i],
        np.log10(g['ecContribution']),
        bins=100
    )
    axs[i].set_xlim(1, 8)
    axs[i].set_xlabel('EC Funding for {} '.format(l) + '(log$_{10}$ EUR)')
    
axs[-1].bar(
    cordis_orgs_df.groupby('activityType')['ecContribution'].sum().index,
    cordis_orgs_df.groupby('activityType')['ecContribution'].sum().values
)
axs[-1].set_xlabel('Institution Type')
axs[-1].set_ylabel('Total Funding (EUR)')
plt.tight_layout()

In [ ]:
activity_type_ohe = pd.get_dummies(cordis_orgs_df['activityType'])

In [ ]:
activity_type_ohe['projectID'] = cordis_orgs_df['projectID']

In [ ]:
project_org_type_counts = activity_type_ohe.groupby('projectID').sum()
org_type_collabs = (project_org_type_counts).T.dot(project_org_type_counts)

In [ ]:
org_type_collabs

In [ ]:
from skbio.diversity.alpha import simpson

## Reports

## Document Similarity

In [ ]:
cordis_reports_df['doc_id'] = cordis_reports_df['projectID'].astype(str) + '_report'
cordis_projects_df['doc_id'] = cordis_projects_df['id'].astype(str) + '_project'

In [ ]:
cordis_reports_df.head(1)

In [ ]:
cordis_reports_df['full_text'] = (cordis_reports_df['summary'].astype(str) 
                                  + cordis_reports_df['workPerformed'].astype(str) 
                                  + cordis_reports_df['article'].astype(str))

In [ ]:
cordis_docs = pd.concat(
    [
        cordis_reports_df[['doc_id', 'full_text']],
        cordis_projects_df[['doc_id', 'objective']].rename(columns={'objective': 'full_text'})
    ],
    axis=0)

In [ ]:
from eu_funding.utils.nlp_utils import remove_markup, normalise_digits
from nesta.packages.nlp_utils.preprocess import clean_and_tokenize, tokenize_document
from gensim.models.phrases import Phrases, Phraser
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.corpora import Dictionary
from scipy.spatial.distance import cosine

In [ ]:
cordis_docs['full_text'] = [normalise_digits(remove_markup(d)) for d in cordis_docs['full_text']]
texts_tokenized = [tokenize_document(d) for d in cordis_docs['full_text']]
texts_tokenized = [list(chain(*a)) for a in texts_tokenized]
bigrams = Phrases(texts_tokenized)
bigrammer = Phraser(bigrams)
texts_bigrammed = bigrammer[texts_tokenized]

In [ ]:
cordis_d2v = Doc2Vec([TaggedDocument(d, [i]) for i, d in enumerate(texts_bigrammed)])

In [ ]:
d2v_test = Doc2Vec([TaggedDocument(d, i) for d, i in zip(texts_bigrammed, ['a', 'b', 'c', 'c', 'd'])])

In [ ]:
cordis_docs['doc_type'] = [s[1] for s in cordis_docs['doc_id'].str.split('_')]
cordis_docs['project_id'] = [s[0] for s in cordis_docs['doc_id'].str.split('_')]

In [ ]:
cordis_docs.reset_index(inplace=True)

In [ ]:
cordis_docs_original = cordis_docs[cordis_docs['doc_type'] == 'project'].set_index('project_id')

In [ ]:
records = []

for i, g in cordis_docs[cordis_docs['doc_type'] == 'report'].groupby('project_id'):
    d2v_ids = g['index']
    if i in cordis_docs_original.index:
        doc_id_project = cordis_docs_original.loc[i]['index']
        d2v_project = cordis_d2v[doc_id_project]
        dists = []
        for d2v_id in d2v_ids:
            d2v_report = cordis_d2v[d2v_id]
            dists.append(cosine(d2v_project, d2v_report))
        records.append({'id': i, 'similarty': np.mean(dists)})

In [ ]:
similarity_df = pd.DataFrame().from_records(records)

In [ ]:
similarity_df['id'] = similarity_df['id'].astype(int)

## Add Fields to Projects

In [ ]:
cordis_projects_df = cordis_projects_df.merge(similarity_df, left_on='id', right_on='id', how='left')

In [ ]:
activity_type_ohe_project = activity_type_ohe.groupby('projectID').sum().reset_index()

In [ ]:
has_activity_type_ohe_project = (activity_type_ohe_project > 0).astype(int)
has_activity_type_ohe_project['projectID'] = activity_type_ohe_project['projectID']
has_activity_type_ohe_project.columns = ['has_{}'.format(a) if a != 'projectID' else 'projectID'
                                         for a in has_activity_type_ohe_project.columns]

In [ ]:
cordis_projects_df = cordis_projects_df.merge(
    activity_type_ohe_project, left_on='id', right_on='projectID', how='left'
)
cordis_projects_df = cordis_projects_df.merge(
    has_activity_type_ohe_project, left_on='id', right_on='projectID', how='left'
)

In [ ]:
cordis_projects_df.to_csv(os.path.join(inter_data_path, 'fp7_h2020_projects.csv'), index=False)

## Geocoding

In [ ]:
import requests

In [ ]:
places = cordis_orgs_df.drop_duplicates('id')[['name', 'street', 'city', 'postCode']]

In [ ]:
addresses = places['name'] + ' ' + places['street'] + ' ' + places['city'] + ' ' + places['postCode']

In [ ]:
def geocode(address):
    params = {
        'address': address,
        'key': api_key
             }
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    r = requests.get(url=base_url, params=params)
    return r.json()

In [ ]:
geos = [geocode(address) for address in addresses]

cordis_orgs_df.head()

In [ ]:
params = {
    'address': '58 Victoria Embankment London',
    'key': api_key
         }
base_url = "https://maps.googleapis.com/maps/api/geocode/json"

r = requests.get(url=base_url, params=params)

In [ ]:
r.json()